# Data Download

### How to Install

1. Install GEE in GCE by following: https://developers.google.com/earth-engine/python_install-conda

  - if encountered jupyter command not found, add conda to path
  
```
    export PATH=~/anaconda3/bin:$PATH
```

2. Make new kernel to use in jupylab

```
conda install ipykernel
ipython kernel install --name ee --user
```

3. Update kernel.json with 3.8.2 python (what gee uses)

```
jupyter kernelspec list
vim /home/cholo/.local/share/jupyter/kernels/ee/kernel.json
```

    - update via vim with this python
  
```
    /home/cholo/anaconda3/envs/ee/bin/python
```

To bypass ee.Authenticate, run 'earthengine authenticate' in terminal

## GEE DL

In [1]:
import sys
sys.path.insert(0, '../utils')

import gee

Enter verification code:  4/ygHq0y3v1k3sUapKuPKGfTPJjhilD5IQgHd4EUQSECx_eZAs4r3Lb9o



Successfully saved authorization token.


In [2]:
BBOX = {
    'arauquita': [-71.69980876899996,6.526376968000022,-70.86773583299998,7.054700505000028], 
#     'cucuta': [-72.60807671699996,7.723856582000053,-72.34649040099998,8.431575791000057], 
    'tibu': [-73.07343705199997,8.251246337000055,-72.47112479799995,9.142041082000048], 
#     'soacha': [-74.30657295399999,4.382138206000036,-74.17129610699999,4.633509322000066], 
#     'villadelrosario': [-72.52267855299993,7.64295204900003,-72.44465188199996,7.902600765000045],
    'saravena': [-72.09962277499994,6.745921904000056,-71.69442685499996,7.0810649060000515]
}

In [3]:
years = [2016, 2017, 2018, 2019, 2020]
areas = ['arauquita', 'tibu', 'saravena']

In [4]:
for area in areas:
    for year in years:
        gee.sen2median(BBOX[area], year, FILENAME = f'gee_{area}_{year}')

Processing gee_arauquita_2016
using COPERNICUS/S2
Task started
Processing gee_arauquita_2017
using COPERNICUS/S2
Task started
Processing gee_arauquita_2018
using COPERNICUS/S2_SR
Task started
Processing gee_arauquita_2019
using COPERNICUS/S2_SR
Task started
Processing gee_arauquita_2020
using COPERNICUS/S2_SR
Task started
Processing gee_tibu_2016
using COPERNICUS/S2
Task started
Processing gee_tibu_2017
using COPERNICUS/S2
Task started
Processing gee_tibu_2018
using COPERNICUS/S2_SR
Task started
Processing gee_tibu_2019
using COPERNICUS/S2_SR
Task started
Processing gee_tibu_2020
using COPERNICUS/S2_SR
Task started
Processing gee_saravena_2016
using COPERNICUS/S2
Task started
Processing gee_saravena_2017
using COPERNICUS/S2
Task started
Processing gee_saravena_2018
using COPERNICUS/S2_SR
Task started
Processing gee_saravena_2019
using COPERNICUS/S2_SR
Task started
Processing gee_saravena_2020
using COPERNICUS/S2_SR
Task started
